In [1]:
import taxcalc as tc
import behresp

%load_ext memory_profiler


In [2]:
def test_func(inplace):
    rec = tc.Records.cps_constructor()
    refyear = 2020
    assert refyear >= 2018
    reform = {'II_em': {refyear: 1500}}
    # ... construct pre-reform calculator
    pol = tc.Policy()
    calc1 = tc.Calculator(records=rec, policy=pol)
    calc1.advance_to_year(refyear)
    # ... construct two post-reform calculators
    pol.implement_reform(reform)
    calc2d = tc.Calculator(records=rec, policy=pol)  # for default behavior
    calc2d.advance_to_year(refyear)

    # Keep track of some of the variables that will be modifed
    # in response if inplace is True.
    df_before_1 = calc1.dataframe(tc.DIST_VARIABLES)
    df_before_2 = calc2d.dataframe(tc.DIST_VARIABLES)

    behresp.response(calc1, calc2d, elasticities={"inc": -0.2}, dump=True, inplace=inplace)

    # Grab the same variables after.
    df_after_1 = calc1.dataframe(tc.DIST_VARIABLES)
    df_after_2 = calc2d.dataframe(tc.DIST_VARIABLES)

    # If inplace is True, the variables should have been modified.
    # If inplace is False, response only modified a copy of calc_1
    # and calc_2.
    if inplace:
        assert not df_before_1.equals(df_after_1)
        assert not df_before_2.equals(df_after_2)
    else:
        assert df_before_1.equals(df_after_1)
        assert df_before_2.equals(df_after_2)


In [3]:
# get a line by line break down of memory usage and changes
# %mprun -f behresp.response test_func(inplace=False)

In [4]:
# obtain high-level stats on memory usage
%memit test_func(inplace=True)

peak memory: 5980.18 MiB, increment: 5805.55 MiB
